In [26]:
#Importing libraries: 
from PIL import Image
import random, os
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from matplotlib import cm

import sys
sys.path.append('/')

import cv2
import numpy as np
from efficientnet_pytorch import EfficientNet
from grad_cam import GradCam,GuidedBackpropReLUModel,show_cams,show_gbs,preprocess_image

## Generate masks for all images in folder:

In [31]:
#Data used to generate the segmentation masks:
source_dir = '/medico2021test'
image_files = os.listdir(source_dir)
#Target folder for final segmentation masks:
target_folder = '/medico2021mask'
#The name of the model:
model_name = '../models/run1.pt'

#Define layer number to extract gradcam heat maps from, depending on run:
if '1' in model_name or '3' in model_name:
    layer_number = '14'
elif '2' in model_name:
    layer_number = '13'
elif '4' in model_name:
    layer_number = '20'
else:
    layer_number = '22'

In [3]:
mask_dic_list = []
for _image in image_files:
    #Load the model
    model = EfficientNet.from_name('efficientnet-b1')
    for params in model.parameters():
        params.requires_grad = True
    #Add extra steps for last layer and correct number of classes (=1):
    in_ftrs = model._fc.in_features
    model._fc = nn.Linear(in_ftrs, 1)
    loaded_model = model
    #Load the model file:
    loaded_model.load_state_dict(torch.load(model_name))
    
    #Generate gradcam:
    image_path = os.path.join(source_dir, _image)
    grad_cam = GradCam(model=loaded_model, blob_name = '_blocks', target_layer_names=[layer_number], use_cuda=False)
    img = cv2.imread(image_path, 1)
    img = np.float32(cv2.resize(img, (224, 224))) / 255
    inputs = preprocess_image(img)
    #Set target as None-> will select target with highest probability
    target_index = None 
    mask_dic = grad_cam(inputs, target_index)
    mask_dic_list.append(mask_dic)

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


In [4]:
#For model with 250 clusters, we use wider thresholds for the segmentation masks:
if '4' in model_name or '5' in model_name:
    segmentation_dic_list = []
    for _dic in mask_dic_list:
        new_dic = _dic[layer_number].copy()
        for _l in range(len(_dic[layer_number])):
            for _p in range(len(_dic[layer_number][_l])):
                if _dic[layer_number][_l][_p] > 0.28 :
                    new_dic[_l][_p] = 1
                else:
                    new_dic[_l][_p] = 0
        new_dic = {layer_number:new_dic}
        segmentation_dic_list.append(new_dic)
else:
    #For model with 50 or 200 clusters (narrow threshold):
    segmentation_dic_list = []
    for _dic in mask_dic_list:
        new_dic = _dic[layer_number].copy()
        for _l in range(len(_dic[layer_number])):
            for _p in range(len(_dic[layer_number][_l])):
                if _dic[layer_number][_l][_p] > 0.4 and _dic[layer_number][_l][_p] < 0.7:
                    new_dic[_l][_p] = 1
                else:
                    new_dic[_l][_p] = 0
        new_dic = {layer_number:new_dic}
        segmentation_dic_list.append(new_dic)
    

## Save the segmentation masks in a folder

In [34]:
for s in range(len(segmentation_dic_list)):
    #Dimensions of original image:
    orig_image = cv2.imread(source_dir + '/' +image_files[s], 1)
    height, width, channels = orig_image.shape
    #Convert the segmentation mask to PIL image
    im = Image.fromarray(np.uint8(cm.gist_earth(segmentation_dic_list[s][layer_number])*255))
    #Resize
    im = im.resize((width,height), Image.ANTIALIAS)
    #Save in target folder:
    im = im.save(target_folder + '/' + image_files[s][:-4] + '.png')